TO DO:
- Figure out what years to restrict data
- Look into mixed data type columns in birding data
- Look into “sensitive species” list
- Sift through tables of interest for FIA
- Figure out how to read in specific tabs in the xlsx files for FIA
- Clean tables to drop extraneous columns
- EDA on data once cleaned
- Consider geospatial uses of data
- Separate observation date into year columm (and maybe month) to group by 
- Value counts on bird species to get top 10 or so to track over years
- Merge birds table with USFWS table to get specific region & download doc off data.gov website



COMPLETED:

<b>X</b> Write a read me

General notes:
- 1st choice: group by wildlife region; else group by county
- try to webscrape polygons off of USFWS website

In [14]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
%matplotlib inline

In [15]:
# reading in sensitive species list
sensitive_species = pd.read_csv('../data/sensitive_species_2000_2020_TN.txt', sep='\t')
sensitive_species = sensitive_species[['GLOBAL UNIQUE IDENTIFIER', 'OBSERVATION DATE', 'TAXONOMIC ORDER', 'CATEGORY', 'COMMON NAME', 'SCIENTIFIC NAME', 'AGE/SEX', 'OBSERVATION COUNT', 'COUNTRY', 'STATE', 'COUNTY', 'IBA CODE', 'BCR CODE', 'USFWS CODE', 'LATITUDE', 'LONGITUDE', 'OBSERVER ID', 'TRIP COMMENTS']]
sensitive_species.columns = ['global_unique_identifier', 'observation_date', 'tax_order', 'category', 'common_name', 'scientific_name', 'age_sex', 'observation_count', 'country', 'state', 'county', 'iba_code', 'bcr_code', 'usfws_code', 'lat', 'long', 'observer_id', 'trip_comments']

# creating additional column for observation year
sensitive_species['observation_year'] = [x[:4] for x in sensitive_species.observation_date]
sensitive_species.head()

,global_unique_identifier,observation_date,tax_order,category,common_name,scientific_name,age_sex,observation_count,country,state,county,iba_code,bcr_code,usfws_code,lat,long,observer_id,trip_comments,observation_year
0,URN:CornellLabOfOrnithology:EBIRD:OBS891731152,2020-03-01,8964,species,Long-eared Owl,Asio otus,NaN,1,United States,Tennessee,Dyer,NaN,26,NaN,36.104154,-89.569913,obsr926054,NaN,2020
1,URN:CornellLabOfOrnithology:EBIRD:OBS865274217,2020-02-03,8964,species,Long-eared Owl,Asio otus,NaN,4,United States,Tennessee,Dyer,NaN,26,NaN,36.104154,-89.569913,obsr167616,NaN,2020
2,URN:CornellLabOfOrnithology:EBIRD:OBS886003275,2020-02-13,8964,species,Long-eared Owl,Asio otus,NaN,3,United States,Tennessee,Dyer,NaN,26,NaN,36.104154,-89.569913,obsr163792,NaN,2020
3,URN:CornellLabOfOrnithology:EBIRD:OBS1031134565,2020-03-01,8964,species,Long-eared Owl,Asio otus,NaN,1,United States,Tennessee,Dyer,NaN,26,NaN,36.104154,-89.569913,obsr910205,NaN,2020
4,URN:CornellLabOfOrnithology:EBIRD:OBS1009893767,2020-11-08,8964,species,Long-eared Owl,Asio otus,NaN,1,United States,Tennessee,Davidson,NaN,24,NaN,36.152192,-86.928407,obsr212351,"Searching for Northern Saw-whet Owl, but had s...",2020


In [16]:
# importing tn counties shape
tn_counties = gpd.read_file('../data/county/tncounty.shp')

NotImplementedError: A polygon does not itself provide the array interface. Its rings do.

In [7]:
# trees TN 2016

# create a dicitonary (with sheet_name =['<sheet1>', '<sheet2>', '<sheet3>'])
# use .get('<sheetname>') to parse into separate dfs

dict_trees2016 = pd.read_excel('../data/TN_2016.xlsx',
             sheet_name = ['Tab 2', 'Tab 8', 'Tab 10', 'Tab 12', 'Tab 24', 'Tab 28', 'Tab 32'])

# cleaning table 2 - land by ownership class
land_by_ownership_class = dict_trees2016.get('Tab 2')
land_by_ownership_class.columns = ['class_type', 'all_forest_land', 'total_unreserved', 'timberland', 'unproductive', 'total_reserved', 'productive', 'unproductive_reserved']
land_by_ownership_class = land_by_ownership_class.head(28)
land_by_ownership_class = land_by_ownership_class.tail(25)
land_by_ownership_class = land_by_ownership_class.set_index('class_type')

In [8]:
# reading in USFWS codes
usfws_codes = pd.read_csv('../data/eBird_2010_to_2014_TN/USFWSCodes.txt', sep='\t')

# cleaning usfws codes df
usfws_codes = usfws_codes.reset_index()
usfws_codes['usfws_name'] = np.where(usfws_codes['USFWS NAME'].isnull(), usfws_codes['USFWS CODE'], usfws_codes['USFWS NAME'])
usfws_codes = usfws_codes[['index','usfws_name']]
usfws_codes.columns = ['usfws_code', 'usfws_name']

In [9]:
# reading in birds 2010 to 2014
birds_2010_2014 = pd.read_csv('../data/eBird_2010_to_2014_TN/eBird_2010_to_2014_TN.txt', sep='\t')

C:\Users\danil\AppData\Local\Temp\ipykernel_38320\1833765151.py:2: DtypeWarning: Columns (11,12,13,23) have mixed types. Specify dtype option on import or set low_memory=False.
  birds_2010_2014 = pd.read_csv('../data/eBird_2010_to_2014_TN/eBird_2010_to_2014_TN.txt', sep='\t')


In [10]:
# cleaning short birds df
short_birds = birds_2010_2014[['GLOBAL UNIQUE IDENTIFIER', 'OBSERVATION DATE', 'TAXONOMIC ORDER', 'CATEGORY', 'COMMON NAME', 'SCIENTIFIC NAME', 'AGE/SEX', 'OBSERVATION COUNT', 'COUNTRY', 'STATE', 'COUNTY', 'IBA CODE', 'BCR CODE', 'USFWS CODE', 'LATITUDE', 'LONGITUDE', 'OBSERVER ID', 'TRIP COMMENTS']]
short_birds.columns = ['global_unique_identifier', 'observation_date', 'tax_order', 'category', 'common_name', 'scientific_name', 'age_sex', 'observation_count', 'country', 'state', 'county', 'iba_code', 'bcr_code', 'usfws_code', 'lat', 'long', 'observer_id', 'trip_comments']

# creating additional column for observation year
short_birds['observation_year'] = [x[:4] for x in short_birds.observation_date]
short_birds.head()

C:\Users\danil\AppData\Local\Temp\ipykernel_38320\3530653956.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  short_birds['observation_year'] = [x[:4] for x in short_birds.observation_date]


,global_unique_identifier,observation_date,tax_order,category,common_name,scientific_name,age_sex,observation_count,country,state,county,iba_code,bcr_code,usfws_code,lat,long,observer_id,trip_comments,observation_year
0,URN:CornellLabOfOrnithology:EBIRD:OBS81849899,2010-01-19,21054,species,American Crow,Corvus brachyrhynchos,NaN,1,United States,Tennessee,Anderson,NaN,28,NaN,35.958338,-84.246758,obsr18758,beautiful warm sunny day,2010
1,URN:CornellLabOfOrnithology:EBIRD:OBS82004086,2010-01-22,21054,species,American Crow,Corvus brachyrhynchos,NaN,5,United States,Tennessee,Anderson,NaN,28,NaN,36.217869,-84.086437,obsr18758,"Chilly, overcast, mist. After not seeing a wi...",2010
2,URN:CornellLabOfOrnithology:EBIRD:OBS80663800,2010-01-02,21054,species,American Crow,Corvus brachyrhynchos,NaN,10,United States,Tennessee,Anderson,NaN,28,NaN,35.996341,-84.163513,obsr54372,All within Haw Ridge Park and within Knoxville...,2010
3,URN:CornellLabOfOrnithology:EBIRD:OBS82003744,2010-01-22,21054,species,American Crow,Corvus brachyrhynchos,NaN,3,United States,Tennessee,Anderson,NaN,28,NaN,36.116386,-84.110001,obsr18758,NaN,2010
4,URN:CornellLabOfOrnithology:EBIRD:OBS287169932,2010-01-15,21054,species,American Crow,Corvus brachyrhynchos,NaN,5,United States,Tennessee,Anderson,NaN,28,NaN,36.007052,-84.262025,obsr276568,NaN,2010


In [12]:
# creating subset of birds sighted within usfws polygon:
birds_usfws = short_birds.loc[~short_birds.usfws_code.isna()]

# joining birds_usfws with usfws codes
birds_usfws = pd.merge(birds_usfws, usfws_codes, on='usfws_code', how='left')

## EDA

In [13]:
short_birds.common_name.value_counts().head(10)

common_name
Northern Cardinal         48842
American Crow             41626
Carolina Chickadee        40837
Blue Jay                  39305
Carolina Wren             39079
Mourning Dove             36616
Tufted Titmouse           36309
American Robin            35295
Northern Mockingbird      31399
Red-bellied Woodpecker    30536
Name: count, dtype: int64

In [ ]:
short_birds.groupby(['observation_year', 'common_name']).count()[['observation_count']]

In [ ]:
# in thousand acres
land_by_ownership_class

### attempting webscraping:

come back to later

In [ ]:
# come back to later!!
# download Selenium webdriver

In [ ]:
# # importing beautiful soup
# import requests
# from bs4 import BeautifulSoup as BS

In [ ]:
# # establishing connection
# URL = 'https://www.fws.gov/refuge/tennessee/map'
# response = requests.get(URL)

In [ ]:
# # checking connection
# response.status_code

In [ ]:
# soup = BS(response.text)

In [ ]:
# soup.findAll('path', attrs={'class':'leaflet-interactive'})